In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bajlecgrausayegh/keras/default/1/config.json
/kaggle/input/bajlecgrausayegh/keras/default/1/metadata.json
/kaggle/input/bajlecgrausayegh/keras/default/1/model.weights.h5
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy


In [85]:
from matplotlib import pyplot as plt
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [133]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, ReLU, Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras import layers, models, regularizers, constraints

In [87]:
x_train = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy")
x_test = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy")
y_train_coarse = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy")

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
import pickle
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [90]:
y_train_fine.shape

(50000,)

In [91]:
x_train.shape

(50000, 32, 32, 3)

In [92]:
x_train,x_val, y_train, y_val = train_test_split(x_train, y_train_fine, test_size=0.1, stratify=y_train_fine)

In [93]:
y_val.shape

(5000,)

In [94]:
x_val.shape

(5000, 32, 32, 3)

In [95]:
x_train_horizontal_flip= x_train[:,:,::-1,:]
x_train_vertical_flip = x_train[:,::-1,:,:]
x_train_vh_flip =x_train[:, ::-1, ::-1, :]

x_train_dup = np.concatenate([x_train,x_train_horizontal_flip,x_train_vertical_flip,x_train_vh_flip],axis=0)
x_train_dup = np.concatenate([x_train,x_train_horizontal_flip],axis=0)
y_train_dup = np.concatenate([y_train, y_train,y_train, y_train], axis=0)
y_train_dup = np.concatenate([y_train, y_train], axis=0)

x_train_bis = x_train_dup.copy()
x_train_bis_bis = x_train_dup.copy()

#desplazamientos right and left 
x_train_bis[:,:,1:,:] =x_train_dup[:,:,:-1,:]
x_train_bis_bis[:,:,:-1,:] =x_train_dup[:,:,1:,:]

x_train_dup = np.concatenate([x_train_dup, x_train_bis, x_train_bis_bis],axis=0)
y_train_dup = np.concatenate([y_train_dup, y_train_dup, y_train_dup], axis=0)


In [96]:
print(f'Original dataset size: {len(x_train)}')
print(f'Augmented dataset size: {len(x_train_dup)}')

Original dataset size: 45000
Augmented dataset size: 270000


In [97]:
x_train_dup.shape, y_train_dup.shape

((270000, 32, 32, 3), (270000,))

In [123]:
from datetime import datetime
import keras

In [147]:
def create_model(input_shape=(32, 32, 3),l2_lambda=0.001):
    # Input layer for images
    inputs = layers.Input(shape=input_shape)
    
    # Flattening layer
    x = layers.Flatten()(inputs)
    
    # Normalization layer
    x = layers.Lambda(lambda x: x/np.float16(255.0))(x)
    
    # Dense layers with L1 regularization
    n = 10
    for i in range(5):
        x = Dense(2**n,
                  kernel_regularizer=regularizers.L2(l2_lambda))(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Dropout(0.15)(x)
        
#     x = layers.Dense(2**(n-2), kernel_regularizer=regularizers.L2(l2_lambda))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
    
    # Output layer with softmax for 100 classes
    fine_output = layers.Dense(100, activation='softmax', name='fine_output')(x)   
  
    # Create model
    model = models.Model(inputs=inputs, outputs=[fine_output], name=f"mlp_allnighter")
    
    return model

print(f"Success at {datetime.now().time()}")

Success at 20:35:23.570353


In [148]:
model = create_model()
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={'fine_output': 'sparse_categorical_crossentropy'},
    metrics={'fine_output': 'accuracy' }
             )

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True,verbose= 1)
tb = TensorBoard(log_dir="logs")

# Display the model summary
model.summary()

Model: "mlp_allnighter"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_23 (Lambda)              │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 1024)           │     3,146,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_69          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_68 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_70          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_69 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_71          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_70 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_72          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_71 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_73          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_72 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 7,468,132 (28.49 MB)

 Trainable params: 7,457,892 (28.45 MB)

 Non-trainable params: 10,240 (40.00 KB)

In [149]:
# Fit the model
history = model.fit(x=x_train_dup,   
                    y={'fine_output': y_train_dup},
                    validation_data=(x_val, {'fine_output': y_val}),
                    epochs=50,                        
                    batch_size=512,  # approximately 1000 batches per epoch                    
                    callbacks=[tb, early_stopping])

Epoch 1/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0955 - loss: 9.5665

I0000 00:00:1730234155.211944      96 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_3', 152 bytes spill stores, 184 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 68 bytes spill stores, 68 bytes spill loads



528/528 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.0956 - loss: 9.5655 - val_accuracy: 0.1806 - val_loss: 8.3173
Epoch 2/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2176 - loss: 7.8849 - val_accuracy: 0.2202 - val_loss: 7.2714
Epoch 3/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2731 - loss: 6.7927 - val_accuracy: 0.2450 - val_loss: 6.4502
Epoch 4/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3164 - loss: 5.9092 - val_accuracy: 0.2406 - val_loss: 5.9175
Epoch 5/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.3566 - loss: 5.1814 - val_accuracy: 0.2478 - val_loss: 5.5421
Epoch 6/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3949 - loss: 4.5894 - val_accuracy: 0.2608 - val_loss: 5.1723
Epoch 7/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.4309 - loss: 4.1101 - val_accuracy: 0.2536 - val_loss: 4.9894
Epoch 8/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.4659 - loss: 3.6994 - val_accuracy: 0.2626 - va

In [102]:
# Save the model
#model.save("accuracy0.2_manylayers_batch_norm2.keras")

In [150]:
predictions = model.predict(x_test).argmax(axis=1)
df = pd.DataFrame(predictions, columns=["Label"])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [151]:
df

,Label
0,69
1,8
2,15
3,51
4,33
...,...
9995,92
9996,21
9997,51
9998,88


In [153]:
df.index.name = "Id"

In [154]:
df.to_csv("submission.csv")